In [1]:
import numpy as np
import scipy 
from scipy import stats
from scipy.stats import norm

In [2]:
def payoff(S,K):
    return np.max(S-K,0)

def Milstein( func, S0 = 100, K = 100, r = 0.0411,
             eta = 1, lam = 1, rho = 0,vmean = 0.1,
             v0 = 0.11,N = 12,n = 100,T = 1):
    
    # N: number of intervals
    # n: number of paths
    # T: maturity
    dt = float(T/N) #interval of each path
    
    W = norm.rvs(size = (n,N))*dt
    Wtilde = norm.rvs(size = (n,N))*dt
 
    B = rho*W + np.sqrt(1-rho**2)*Wtilde
    
    V = np.zeros((n,N+1)) # V is our variance 
    V[:,0] = v0
    S = np.zeros((n,N+1)) # S is our stock price
    S[:,0] = S0
    
    
    for i in range(1,N+1):
        V[:,i] = V[:,i-1] - lam*(V[:,i-1] -vmean)*dt + eta*np.sqrt(V[:,i-1])*B[:,i-1]+0.25*eta**2*(B[:,i-1]**2 - dt)
        V[:,i] = np.abs(V[:,i] )
        
    dlogS = (r - 0.5*V[:,1:])*dt + np.sqrt(V[:,1:])*W
    dlogS_sum = np.cumsum(dlogS,axis = 1)
    S[:,1:] = S0*np.exp(dlogS_sum)
    
    price = np.mean(func(S[:,N],K))
   
    return price

Milstein(payoff)

18.92496756760282

In [ ]:

import numpy as np
from scipy.stats import norm
from scipy.optimize import minimize

mkt_price = np.array([[6.5757, 2.8223, 0.6335], [8.1165, 4.3850, 1.7263],
                      [6.0865, 3.1820, 1.2317], [7.7710, 4.7369, 2.4165]])


T = np.repeat(np.array([[0.25], [0.5], [0.75], [1]]), 3, axis=1)
K = np.array([np.arange(95, 110, 5), np.arange(95, 110, 5), np.arange(100, 115, 5), np.arange(100, 115, 5)])

mkt_price = mkt_price.reshape(-1,1)
T = T.reshape(-1,1)
K = T.reshape(-1,1)
def SV_loss_fun(params,S, T, K, r=0.0411, mkt = mkt_price.reshape(12)):
    eta = params[0]
    lam = params[1]
    rho = params[2]
    vmean = params[3]
    v0 = 0.2
    N = 1000
    n = 100
    
    loss = 0
    for i in range(len(T)):
        loss += (Milstein(payoff,S,K[i],r,eta,lam,rho,vmean,v0,N,n,T[i]) - mkt[i])**2
    
    
    
    return loss

def SV_Calibrate(S,T,K,r = 0.0411):
    result = minimize(SV_loss_fun,Input,(S,T,K),method ='COBYLA')
    return result


Input = [1,1,0.5,0.1]
parameters = SV_Calibrate(100,T,K)
print('eta: ',parameters.x[0],', init: ',Input[0])
print('lam: ',parameters.x[1],', init: ',Input[1])
print('rho: ',parameters.x[2],', init: ',Input[2])
print('mean: ',parameters.x[3],', init: ',Input[3])



/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:16: RuntimeWarning: invalid value encountered in sqrt
  app.launch_new_instance()



CG: